# [Pyomo.GDP](./index.ipynb) Logical Expression System Demo - Process Network Example

This demo illustrates how to work with Pyomo.GDP logical expressions within the context of a process network.

> Demo incomplete. Work in progress.

This code relies on the logic-v1 branch at https://github.com/qtothec/pyomo/tree/logic-v1

In [1]:
from pyomo.environ import *
from pyomo.gdp import *
from pyomo.core.expr.logical_expr import *
from pyomo.core.plugins.transform.logical_to_linear import update_boolean_vars_from_binary

In [2]:
m = ConcreteModel()
m.units = RangeSet(5)
m.components = Set(initialize=['A', 'B'])
m.streams = Set(initialize=[(1, 3), (1, 4), (1, 5), (2, 3), (2, 4), (2, 5)])
m.fc = Var(m.streams, m.components, domain=NonNegativeReals, doc="Component flow")

stream_to_inlet_unit = {1: 3, 2: 4, 3: 5, 4: 3, 5: 4, 6: 5}
stream_to_outlet_unit = {1: 1, 2: 1, 3: 1, 4: 2, 5: 2, 6: 2}

m.use_unit = Disjunct(m.units)
m.no_unit = Disjunct(m.units)

@m.Disjunction(m.units)
def unit_exists_or_absent(m, unit):
    return [m.use_unit[unit], m.no_unit[unit]]

m.Yunit = BooleanVar(m.units)
for unit in m.units:
    m.Yunit[unit].set_binary_var(m.use_unit[unit].indicator_var)
    
@m.Disjunct(m.streams)
def stream_exists(disj, u1, u2):
    # This is the disjunct for stream s existing
    pass

@m.Disjunct(m.streams)
def no_stream(disj, u1, u2):
    @disj.Constraint(m.components)
    def no_flow(disj, c):
        return m.fc[u1, u2, c] == 0

@m.Disjunction(m.streams)
def stream_exists_or_not(m, u1, u2):
    return [m.stream_exists[u1, u2], m.no_stream[u1, u2]]

m.Ystream = BooleanVar(m.streams)
for s in m.streams:
    m.Ystream[s].set_binary_var(m.stream_exists[s].indicator_var)

@m.LogicalStatement(m.streams)
def unit_exists_iff_connected(m, u1, u2):
    return m.Ystream[u1, u2].equivalent_to(m.Yunit[u1] & m.Yunit[u2])